## Step 1: Hello Data 

In [ ]:
import pandas as pd

# Option 1: Use raw string with r prefix
file_path = r"D:\Applied AI & ML\Machine Learning Porgramming\Labs\ecommerce_500_rows (1).csv"

# Option 2: Use double backslashes (\\)

df = pd.read_csv(file_path)
df.head(3)
# test
# test 2

,Order Date,Customer ID,Product,Unit Price,Quantity,Coupon Code,City
0,05/25/2023,CUST1000,Keyboard,533.07,4,NaN,Calgary
1,11/21/2023,CUST1001,Monitor,1429.51,5,SAVE20,Vancouver
2,06/28/2023,CUST1002,Smartphone,1139.69,4,NaN,Montreal


## Step 2: Pick the Right Container

Since it enables the encapsulation of both data and functionality (such as clean/total functions), a class is the ideal containment for transaction information.

# Quick profiling 

In [2]:


from dataclasses import dataclass
from typing import Optional
from datetime import datetime

@dataclass
class Transaction:
    date: str
    customer_id: str
    product: str
    price: float
    quantity: int
    coupon_code: Optional[str]
    shipping_city: str

    def clean(self):
        if isinstance(self.price, str) and self.price == "N/A":
            self.price = 0.0
        if self.price < 0:
            self.price = abs(self.price)

    def total(self):
        return self.price * self.quantity
